Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame
Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [3]:
from pyspark.sql.functions import sum, avg, when, max
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [4]:
spark = SparkSession.builder.master('local[*]').appName('window_pivot').getOrCreate()

In [5]:
data = [("2023-11-20", "Electronics", 100, 12000),
        ("2023-11-21", "Electronics", 110, 13000),
        ("2023-11-22", "Electronics", 105, 12500),
        ("2023-11-20", "Clothing", 300, 15000),
        ("2023-11-21", "Clothing", 280, 14000),
        ("2023-11-22", "Clothing", 320, 16000),
        ("2023-11-20", "Books", 150, 9000),
        ("2023-11-21", "Books", 200, 12000),
        ("2023-11-22", "Books", 180, 10000)]
colms = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, colms)

In [52]:
#С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта
windowSpec = Window.partitionBy("category")
df_avg = df.withColumn("avg_revenue", F.round(avg("revenue").over(windowSpec)))
df_avg_distinct = df_avg.select('category', 'avg_revenue').distinct()

In [53]:
df_avg_distinct.show()

+-----------+-----------+
|   category|avg_revenue|
+-----------+-----------+
|      Books|    10333.0|
|   Clothing|    15000.0|
|Electronics|    12500.0|
+-----------+-----------+



In [14]:
#Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом,
#чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день
df_pivot = df.groupBy("category").pivot("date").agg(F.avg("revenue"))
df_pivot.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+



In [17]:
#просто другой способ (транспонированная таблица)
df_pivot2 = df.groupBy("date").pivot("category").agg(F.avg("revenue"))

In [18]:
df_pivot2.show()

+----------+-------+--------+-----------+
|      date|  Books|Clothing|Electronics|
+----------+-------+--------+-----------+
|2023-11-20| 9000.0| 15000.0|    12000.0|
|2023-11-21|12000.0| 14000.0|    13000.0|
|2023-11-22|10000.0| 16000.0|    12500.0|
+----------+-------+--------+-----------+

